# Multi Crypto Strategy with multiple algorithms


In [ ]:
from config import crypto
from config import general as config
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from lib.drl import load_dataset, data_split
from lib.stocks_strategy import StocksStrategy
from lib.support import check_directory_structure, get_time, get_duration, log_duration
import pandas as pd
import time

ROOT_DIR = '.'
check_directory_structure(ROOT_DIR)

FILE_PREFIX = "crypto10"

In [ ]:
# df = load_dataset(f'{ROOT_DIR}/datasets/crypto/crypto10_5min.csv', crypto.INDICATORS, use_turbulence=True, use_vix=False, time_interval='5min')
# df.to_csv(f'{ROOT_DIR}/datasets/crypto/crypto10_5min_parsed.csv')

In [ ]:
df = pd.read_csv(f"{config.DATA_SAVE_DIR}/crypto/{FILE_PREFIX}_1h_parsed.csv", index_col=0)
train_df = data_split(df, crypto.TRAIN_START_DATE, crypto.TRAIN_END_DATE)
test_df = data_split(df, crypto.TEST_START_DATE, crypto.TEST_END_DATE)
print(f"train {train_df.shape} start: {crypto.TRAIN_START_DATE} end: {crypto.TRAIN_END_DATE}")
print(f"test  {test_df.shape} start: {crypto.TEST_START_DATE} end: {crypto.TEST_END_DATE}")

In [ ]:
stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2 * stock_dimension + len(crypto.INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

## Train and Testing a single algorithm

In [ ]:
def get_train_env():
    env_kwargs = {
      "hmax": 100,
      "initial_amount": 1000000,
      "num_stock_shares": num_stock_shares,
      "buy_cost_pct": buy_cost_list,
      "sell_cost_pct": sell_cost_list,
      "state_space": state_space,
      "stock_dim": stock_dimension,
      "tech_indicator_list": crypto.INDICATORS,
      "action_space": stock_dimension,
      "reward_scaling": 1e-4
    }
    e_train_gym = StockTradingEnv(df=train_df, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    return env_train

In [ ]:
stockStrategy = StocksStrategy(model_name="SAC", root_dir=ROOT_DIR, results_dir=config.RESULTS_DIR, trained_model_dir=config.TRAINED_MODEL_DIR, prefix='crypto10')

In [ ]:
# # TRAIN
start = time.time()
total_timesteps = 50000
stockStrategy.train(get_train_env(), total_timesteps, use_existing=False)
log_duration(start)

In [ ]:
# # test
start = time.time()
e_trade_gym = StockTradingEnv(df=test_df, turbulence_threshold=70, **env_kwargs)
stockStrategy.test(e_trade_gym)
log_duration(start)

## Training and Testing multiple algorithms at once

In [ ]:
model_list = [ "DDPG", "A2C", "PPO", "TD3"] #"SAC",

In [ ]:
main_start = time.time()
for model_name in model_list:
    start = time.time()
    stockStrategy = StocksStrategy(model_name, root_dir=ROOT_DIR, results_dir=config.RESULTS_DIR, trained_model_dir=config.TRAINED_MODEL_DIR, prefix='crypto10')
    print(model_name)
    total_timesteps = 50000 # 500
    stockStrategy.train(env_train, total_timesteps, use_existing=True)
    log_duration(start)
log_duration(main_start)

In [ ]:
main_start = time.time()
for model_name in model_list:
    start = time.time()
    e_trade_gym = StockTradingEnv(df=trade_df, turbulence_threshold=70, risk_indicator_col='vix', **env_kwargs)
    stockStrategy = StocksStrategy(model_name, root_dir=ROOT_DIR, results_dir=config.RESULTS_DIR, trained_model_dir=config.TRAINED_MODEL_DIR)
    stockStrategy.test(e_trade_gym)
    log_duration(start)
log_duration(main_start)